In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas

iuc = pd.read_csv("C:/Users/lch/Desktop/iuc2.csv",header=0)

pn.extension()
#设置图形类型、position范围、值的类型、是否进行pvalue筛选、log2foldchange范围的调节模块
picfilter = pn.widgets.RadioButtonGroup(name='picture filter', options=['scatter', 'heatmap'],width=200)
x = pn.widgets.RangeSlider(name='codons Range', start=1, end=250,step = 1,width=200)
y = pn.widgets.RadioButtonGroup(name='Button Group', options=['pvalue', 't3_vs_t0'],width=200)
pfilter = pn.widgets.RadioButtonGroup(name='pvalue filter', options=["original",'pvalue<0.05'],width=200)
cfilter = pn.widgets.RangeSlider(name='change filter', start=min(round(iuc["t3_vs_t0"])), end=max(round(iuc["t3_vs_t0"])),step = 0.5,width=200)

#通过@pn.depends来实时控制图形与表格
@pn.depends(x,y,pfilter,cfilter,picfilter,watch=True)
def pic(x,y,pfilter,cfilter,picfilter):
    data = iuc[iuc["position"] <= int(x[1])]
    data = data[data["position"] >= int(x[0])][data["t3_vs_t0"]<= int(cfilter[1])][data["t3_vs_t0"]>= int(cfilter[0])]
    if pfilter=="original":
        pass
    else:
        data = data[data["pvalue"] <= 0.05]
    if picfilter=="heatmap":#输出热图
        plot = data.hvplot.heatmap(x="position", y="amino_mut",title=y+' of codon change'+"("+pfilter+")", xaxis='top',rot=70,legend = "left",hover=True,hover_cols=["amino_wt","pvalue"],
                               cmap=["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"],
                               width=1250, height=500, C=y).opts(toolbar=None,fontsize={'title': 10, 'xticks': 8, 'yticks': 8})
    else:#输出火山图
        result = pd.DataFrame()
        result['x'] = data['t3_vs_t0']
        result['y'] = data['pvalue']
        result['-log10(pvalue)'] = -data['pvalue'].apply(np.log10)
        result['position'] = data['position']
        result['amino_wt'] = data['amino_wt']
        result['amino_mut'] = data['amino_mut']

        # 设置显著性阈值
        x_threshold = 1
        y_threshold = 0.00001

        # 分组为up, normal, down
        result['group'] = 'nan'
        result.loc[(result.x > x_threshold) & (result.y < y_threshold), 'group'] = 'up'
        result.loc[(result.x < -x_threshold) & (result.y < y_threshold), 'group'] = 'down'
        result.loc[(result.x >= -x_threshold) & (result.x <= x_threshold) | (result.y >= y_threshold), 'group'] = 'ns'
        plot =result.hvplot(x='x', y='-log10(pvalue)', color='group',
                      hover_cols=["position", "amino_wt", "amino_mut"],
                      cmap={'up': 'red', 'down': 'blue', 'ns': 'dimgrey', 'nan': 'black'}, kind='scatter')
    return plot

#将图中显示数据通过表格的方式显示出来
@pn.depends(x,y,pfilter,cfilter,watch=True)
def cssv(x,y,pfilter,cfilter):
    data = iuc[iuc["position"] <= int(x[1])]
    data = data[data["position"] >= int(x[0])][data["t3_vs_t0"] <= int(cfilter[1])][data["t3_vs_t0"] >= int(cfilter[0])]
    if pfilter == "original":
        pass
    else:
        data = data[data["pvalue"] <= 0.05]
    return data.hvplot.table(title = 'DATAtable' )


#利用panel的快速模板做一个承载上述代码的页面
pn.template.FastListTemplate(
    site="team5_homework", title="find_mutant",
    sidebar=["Options", pn.WidgetBox(picfilter,x, y,pfilter,cfilter)],
    main=["plot and table",pic,cssv]).show()
